In [1]:
import pickle 
import pandas as pd 
import seaborn as sns
import numpy as np 
import json
import itertools
import lifelines
import glob
import os
import copy
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.cm as cm
from matplotlib.colors import Normalize, LinearSegmentedColormap
import matplotlib.patches as patches
from matplotlib.patches import Patch, Rectangle
from matplotlib.lines import Line2D
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test
import sys
import scipy.stats

my_path = "/Users/jdf36/Desktop/bwh/cancer_regions/src/"

sys.path.insert(1, my_path)
from jenks import load_clinvar_df, getJenksBreaks, altered_generate_coding_position_list
from utils import place_position
from survival_tools import (
    annotate_region_diagram_ax, 
    get_km_curve_ax,
    risk_ratio,
    get_pfam_domains,
    get_single_gene_and_breaks,
    make_hr_lr_binary_diagram,
    get_colors,
    position_in_pfam_lists,
    get_pfam_domains,
    get_annotated_pfam_profiles,
    get_logrank_p,
    get_annotated_hr_lr_regions,
    get_pfam_comparison_logrank_vals
)
from annotate_funtional_assays import get_functional_from_key
from cosmic import get_cosmic_ratios, get_gene_cosmic, get_cosmic_ratios_altered
from config import gene_to_end_position


In [2]:
file = open("../gene_to_df_delet_removed_092922.pickle",'rb')
gene_to_df = pickle.load(file)
logrank_df = pd.read_csv("../all_logrank_df_092922.csv")
regression_results = pd.read_csv("../regression_results_delet_removed_092922.csv", index_col = 0)
file = open("../mega_results_checkppoint_092922.pickle",'rb')
mega_file = pickle.load(file)

In [3]:
genes_for_diagram = ["ATM", "BRCA1", "BRCA2", "CHEK2", "MSH6", "PALB2", "TP53"]

rows = []

row_val_d = copy.deepcopy(mega_file["ATM"][2][2])
del row_val_d["Region"]
del row_val_d["carriers"]
row_keys = "|".join(row_val_d)

for k, v in mega_file.items():
    if k not in genes_for_diagram:
        continue
    for k2, v2 in v.items():
        # k2 is num breaks
        for k3,v3 in v2.items():
            #k3 is the regional info 
            if type(k3) == int:
                ## can't publish n carriers per UKB data agreement 
                v3_c = copy.deepcopy(v3)
                region_num = k3
                n_breaks = k2
                breakpoints = v2["breakpoints"]
                breakpoints[-1] = gene_to_end_position[k]
                breakpoints_str = "|".join(list(map(lambda x  : str(x), breakpoints)))
                if "carriers" in v3_c.keys():
                    del v3_c["carriers"]
                    
                if "Region" in v3_c.keys():
                    del v3_c["Region"]
                k_list = "|".join(list(map(lambda x : str(x), v3_c.keys())))
                assert k_list == row_keys
                vals = "|".join(list(map(lambda x : str(x), v3_c.values())))
                rows.append([k, n_breaks, region_num, breakpoints, vals])
                

supp_df = pd.DataFrame(rows, columns = ["Gene", "Number of Breaks", "Region", "Breakpoints", row_keys])
supp_df = supp_df.sort_values(by = ["Gene", "Number of Breaks", "Region"])
supp_df.to_csv("gene_region_measurements_all_break_numbers_7_genes.csv")

In [4]:
logrank_df

,key,p,gene,breaks,hr_diff,chisq_p,breakpoints,high_regions,low_regions
0,BRCA2_3_0.05,0.095513,BRCA2,3,0.05,0.236247,0|3244|6730|10234.0,3,2
1,BRCA2_4_0.05,0.046432,BRCA2,4,0.05,0.227904,0|2020|4591|7266|10234.0,2|4,3
2,BRCA2_5_0.05,0.040333,BRCA2,5,0.05,0.278038,0|1970|4366|6586|8245|10234.0,2|5,3
3,BRCA2_6_0.05,0.004780,BRCA2,6,0.05,0.086249,0|1286|2881|4707|6656|8295|10234.0,3|6,1|4
4,BRCA2_7_0.05,0.035203,BRCA2,7,0.05,0.478008,0|1219|2748|4474|6100|7375|8548|10234.0,2|5|7,1|4
...,...,...,...,...,...,...,...,...,...
561,TP53_11_0.35,0.000735,TP53,11,0.35,0.127361,0|273|427|493|537|613|672|764|826|857|920|1025.0,2|7|8,1|4|6
562,TP53_12_0.35,0.000404,TP53,12,0.35,0.138879,0|273|427|493|537|613|672|725|772|826|857|920|...,2|7|9,1|4|6|8
563,TP53_13_0.35,0.004613,TP53,13,0.35,0.443436,0|158|377|437|493|537|613|672|725|772|826|857|...,2|3|8|10,1|5|7|9
564,TP53_14_0.35,0.003426,TP53,14,0.35,0.508779,0|158|329|404|455|493|537|613|672|725|772|826|...,2|3|9|11,1|4|6|8|10


In [5]:
pd.read_csv("mutscore_0.66_0.33_high_low_survival_scores.csv")

,Unnamed: 0,gene,logrank p,n_high_carriers,n_low_carriers
0,0,ABRAXAS1,0.210108,24,239
1,1,AKT1,0.843938,42,261
2,2,ATM,0.825953,1846,3258
3,3,BABAM2,0.804963,30,368
4,4,BARD1,0.341654,94,1523
5,5,BRCA1,0.214774,109,2618
6,6,BRCA2,0.302055,576,6243
7,7,BRIP1,0.066065,212,1560
8,8,CDH1,0.547029,125,1479
9,9,CHEK2,0.326771,313,832


In [22]:
using_p = logrank_df.loc["TP53_15_0.1"]["p"]
gene_df = logrank_df.loc[logrank_df["gene"] == "TP53"].sort_values(by = "p")

gene_df.iloc[0]

p                                                     0.000306741
gene                                                         TP53
breaks                                                         12
hr_diff                                                       0.2
chisq_p                                                  0.138879
breakpoints     0|273|427|493|537|613|672|725|772|826|857|920|...
high_regions                                                2|7|9
low_regions                                            1|4|6|8|11
Name: TP53_12_0.2, dtype: object

In [24]:
sorted_logrank = logrank_df.sort_values(by = "p")

counter = 0
m = len(sorted_logrank)
alpha = 0.01
signif_genes = set()
for index, row in sorted_logrank.iterrows():
    counter += 1
    if row["p"] <= (counter/m)*alpha:
        signif_genes.add(row["gene"])
        
signif_genes

{'ATM', 'BRCA1', 'BRCA2', 'CHEK2', 'MSH6', 'PALB2', 'TP53'}

In [13]:
logrank_df

,p,gene,breaks,hr_diff,chisq_p,breakpoints,high_regions,low_regions
key,,,,,,,,
BRCA2_3_0.05,0.095513,BRCA2,3,0.05,0.236247,0|3244|6730|10234.0,3,2
BRCA2_4_0.05,0.046432,BRCA2,4,0.05,0.227904,0|2020|4591|7266|10234.0,2|4,3
BRCA2_5_0.05,0.040333,BRCA2,5,0.05,0.278038,0|1970|4366|6586|8245|10234.0,2|5,3
BRCA2_6_0.05,0.004780,BRCA2,6,0.05,0.086249,0|1286|2881|4707|6656|8295|10234.0,3|6,1|4
BRCA2_7_0.05,0.035203,BRCA2,7,0.05,0.478008,0|1219|2748|4474|6100|7375|8548|10234.0,2|5|7,1|4
...,...,...,...,...,...,...,...,...
TP53_11_0.35,0.000735,TP53,11,0.35,0.127361,0|273|427|493|537|613|672|764|826|857|920|1025.0,2|7|8,1|4|6
TP53_12_0.35,0.000404,TP53,12,0.35,0.138879,0|273|427|493|537|613|672|725|772|826|857|920|...,2|7|9,1|4|6|8
TP53_13_0.35,0.004613,TP53,13,0.35,0.443436,0|158|377|437|493|537|613|672|725|772|826|857|...,2|3|8|10,1|5|7|9
